In [2]:
from google.colab import files
uploaded = files.upload()

Saving ptbxl_database.csv to ptbxl_database.csv


In [3]:
from google.colab import files
uploaded = files.upload()

Saving scp_statements.csv to scp_statements.csv


In [4]:
from google.colab import files
uploaded = files.upload()

Saving Lab Session Data.xlsx to Lab Session Data.xlsx


In [5]:
from google.colab import files
uploaded = files.upload()

Saving 01000_lr.dat to 01000_lr.dat
Saving 01000_lr.hea to 01000_lr.hea
Saving 01001_lr.dat to 01001_lr.dat


In [6]:
#A1
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Load the files
ptbxl_df = pd.read_csv("ptbxl_database.csv")
scp_df = pd.read_csv("scp_statements.csv")

# Filter diagnostic codes
diagnostic_codes = scp_df[scp_df['diagnostic'] == 1.0]['Unnamed: 0'].tolist()

# Convert scp_codes string to dictionary
ptbxl_df['scp_codes'] = ptbxl_df['scp_codes'].apply(ast.literal_eval)

# Extract diagnostic codes from each row
ptbxl_df['diagnostic_codes'] = ptbxl_df['scp_codes'].apply(
    lambda x: [code for code in x if code in diagnostic_codes]
)

# Define MI-related codes
mi_labels = {'IMI', 'ASMI', 'AMI', 'ALMI', 'ILMI'}

# Label ECGs as NORM or MI
def get_binary_label(codes):
    if 'NORM' in codes:
        return 'NORM'
    elif any(code in mi_labels for code in codes):
        return 'MI'
    return None

ptbxl_df['label'] = ptbxl_df['diagnostic_codes'].apply(get_binary_label)

# Filter only binary classes
binary_df = ptbxl_df.dropna(subset=['label'])

# Prepare features (example using age and weight — replace with actual signal data for real model)
X = binary_df[['age', 'weight']].fillna(0)
y = binary_df['label']

# Encode labels
y = y.map({'NORM': 0, 'MI': 1})

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# Train kNN
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

# Predict
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluate
print("Training Data Classification Report:")
print(classification_report(y_train, y_train_pred))

print("Test Data Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred))


Training Data Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.74      0.76      6659
           1       0.57      0.65      0.61      3564

    accuracy                           0.71     10223
   macro avg       0.68      0.69      0.68     10223
weighted avg       0.72      0.71      0.71     10223

Test Data Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.71      0.73      2855
           1       0.52      0.58      0.55      1527

    accuracy                           0.67      4382
   macro avg       0.64      0.65      0.64      4382
weighted avg       0.68      0.67      0.67      4382

Confusion Matrix (Test):
[[2028  827]
 [ 639  888]]
